In [224]:
import pandas as pd

In [229]:
file_path = '..\\data\\JPMX_hardcopy.xlsx'
df_raw = pd.read_excel(file_path, header=[0,1]) # Skip the first title row
df_raw = df_raw.dropna(axis=0, how='all').reset_index(drop=True)
clean_data = {}
clean_data['Date'] = df_raw.iloc[:, 0]

In [231]:
for i in range(1, df_raw.shape[1], 2):
    header_part1 = df_raw.columns[i][0]
    header_part2 = df_raw.columns[i][1]

    if header_part2.endswith('.1'):
        header_part2 = header_part2[:-2]
    
    new_col_name = f"{header_part1}_{header_part2}".strip().replace(' ', '_')
    
    clean_data[new_col_name] = df_raw.iloc[:, i]

df = pd.DataFrame(clean_data)
df

,Date,EMBIG_Div_JPGCSOSD_Index,High_Grade_JPGCIGSS_Index,High_Yield_JPGCHYSS_Index,Africa_JPSSGDAF_Index,Asia_JPSSGDAS_Index,Europe_JPSSGDER_Index,Latin_America_JPSSGDLA_Index,Middle_East_JPSSGDME_Index
0,2005-01-03,371.522,150.318,521.822,286.0,253.0,215.0,486.0,333.0
1,2005-01-04,370.689,147.913,522.278,273.0,248.0,207.0,492.0,325.0
2,2005-01-05,378.834,153.579,532.467,274.0,247.0,217.0,503.0,324.0
3,2005-01-06,382.809,154.331,539.133,276.0,250.0,221.0,508.0,326.0
4,2005-01-07,380.195,150.906,537.202,272.0,248.0,216.0,507.0,323.0
...,...,...,...,...,...,...,...,...,...
5134,2025-07-24,302.167,106.920,520.577,430.0,101.0,262.0,383.0,254.0
5135,2025-07-25,302.322,103.293,512.556,429.0,97.0,261.0,384.0,254.0
5136,2025-07-28,296.832,108.334,519.836,418.0,103.0,253.0,380.0,250.0
5137,2025-07-29,302.588,105.066,515.766,428.0,98.0,261.0,383.0,256.0
